In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import nltk    
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import re
import string
stopwords= stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Ujjval
[nltk_data]     Priyadarshi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from keras.optimizers import Adam
from nltk.tokenize import word_tokenize

In [6]:
dt= pd.read_csv('train.csv')

<h1><b> Data Cleaning </b></h1>
Here we are gonna clean the DF. Specifically, we clean:

stopwords (Kept cause removing them cause drop of performances)
<li>URL</li>
<li>HTML</li>
<li>emoji</li>
<li>punctuation</li>

In [7]:
def clean_df(df):
    def remove_stopwords(text):
        if text is not None:
            tokens = [x for x in word_tokenize(text) if x not in stop]
            return " ".join(tokens)
        else:
            return None
       
    df["text"] = df['text'].apply(lambda x : x.lower())
    
    
    def remove_URL(text):
        url = re.compile(r'https?://\S+|www\.\S+')
        return url.sub(r'',text)
    
    df['text']=df['text'].apply(lambda x : remove_URL(x))
    
    
    def remove_html(text):
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)
    
    df['text']=df['text'].apply(lambda x : remove_html(x))
    
    
    def remove_emoji(text):
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    df['text']=df['text'].apply(lambda x: remove_emoji(x))
    
    
    def remove_punct(text):
        table=str.maketrans('','',string.punctuation)
        return text.translate(table)

    df['text']=df['text'].apply(lambda x : remove_punct(x))
    
    
    df.text = df.text.replace('\s+', ' ', regex=True)
    return df  

In [8]:
df = clean_df(dt)

In [9]:
list_corpus = df["text"].tolist()
list_labels = df["target"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=40)

In [24]:
len(X_train), len(padded)

(6090, 6090)

In [10]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(X_train)
padded = pad_sequences(sequences,maxlen=max_length, truncating='post')

In [11]:
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_____________________

In [13]:
num_epochs = 10
history = model.fit(padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test))

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
6090/6090 [==============================] - 2s 345us/sample - loss: 0.6820 - acc: 0.5716 - val_loss: 0.6820 - val_acc: 0.5594
Epoch 2/10
6090/6090 [==============================] - 1s 142us/sample - loss: 0.6711 - acc: 0.5731 - val_loss: 0.6639 - val_acc: 0.5594
Epoch 3/10
6090/6090 [==============================] - 1s 140us/sample - loss: 0.6262 - acc: 0.6415 - val_loss: 0.5862 - val_acc: 0.7544
Epoch 4/10
6090/6090 [==============================] - 1s 132us/sample - loss: 0.5133 - acc: 0.7990 - val_loss: 0.5060 - val_acc: 0.7663
Epoch 5/10
6090/6090 [==============================] - 1s 154us/sample - loss: 0.4185 - acc: 0.8356 - val_loss: 0.4535 - val_acc: 0.8063
Epoch 6/10
6090/6090 [==============================] - 1s 139us/sample - loss: 0.3647 - acc: 0.8557 - val_loss: 0.4474 - val_acc: 0.8050
Epoch 7/10
6090/6090 [==============================] - 1s 152us/sample - loss: 0.3247 - acc: 0.8718 - val_loss: 0.4335 - va

In [38]:
pred_dp= model.predict(testing_padded)

In [39]:
output = []
for val in pred_dp:
    if val > 0.5:
        output.append(1)
    else:
        output.append(0)

In [40]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [41]:
accuracy_score(output, y_test)

0.8154957321076822

In [42]:
confusion_matrix(output, y_test)

array([[736, 165],
       [116, 506]], dtype=int64)

In [43]:
dt_test= pd.read_csv('test.csv')

In [44]:
dt_test = clean_df(dt_test)

In [45]:
test_corpus= dt_test['text']

In [46]:
sequences_test = tokenizer.texts_to_sequences(test_corpus)
padded_test = pad_sequences(sequences_test,maxlen=max_length, truncating='post')

In [47]:
pred= model.predict(padded_test)

In [48]:
final_output = []
for val in pred:
    if val > 0.5:
        final_output.append(1)
    else:
        final_output.append(0)

In [49]:
len(dt_test)

3263

In [50]:
s= pd.read_csv("sample_submission.csv")

submission = pd.DataFrame({
        "id": s["id"],
        "target": final_output
    })

submission.to_csv('submission_nlp.csv', index=False)  